# 59: Identifying Biases in Criminal Sentencing
##### Priyanka Adhikari, Ruchira Banerjee, Nidhi Bendre, Simone Kaplunov

## Executive Summary

Our study aimed to examine biases in the court system and criminal sentencing. Although our model had some errors, including outliers and skewed coefficients, we found [evidence](#graphs) suggesting that defendants of certain races tended to receive higher prison sentences, indicating a potential bias (See Figure 1). Similarly, certain citizenship statuses also corresponded to higher average prison times (See Figure 2). We also observed that the type of crime committed was a significant factor in determining prison time, which aligns with expectations (See figure 4). However, because our dataset was [biased](#bias), the results from our project [should not be used to make predictions](#takeaway) and should only be used to highlight biases in the system. 

## Introduction

A large issue in criminal proceedings stems from possible biases in court sentencing procedures. There have been many allegations of instances where defendants of certain races, ethnicities, genders, etc., have faced longer sentences than others, despite similarly committing the same crime. The central motivation of this project is to identify potential disparities in sentencing outcomes for individuals of different races.

As court proceedings are notoriously complex, the model will not fully encompass all the necessary factors needed to predict a defendant's prison sentence. Still, it serves as a baseline for future work in developing models and artificial intelligence systems to try and eradicate biases in criminal prison sentencing procedures by offering a less-biased method to determine a defendant's prison sentence.

## Data Description

Our dataset contained the statistics of 75,000 defendants in the year of 2018 to 2019. 

This federal criminal database was obtained from the [National Archive of Criminal Justice Data](https://www.icpsr.umich.edu/web/ICPSR/studies/37990/datadocumentation#) to observe various features of a given defendant. In our analysis of this data, we used predominantly categorical variables.

These included:
- <b> AGE</b>: the defendant's age at the time of their sentencing
- <b> CITIZEN</b>: the nature of the defendant's citizenship
- <b> MONRACE</b>: the defendant's race
- <b> HISPORIG</b>: the defendant's ethnic origin
- <b> MONSEX</b>: the defendant's gender
- <b> TOTPRISN</b>: the total months of imprisonment ordered
- <b> OFFGUIDE</b>: the primary type of crime for the case
- <b> NOCOUNTS</b>: the number of counts of conviction
- <b> TOTCHPTS</b>: the total number of criminal history points applied
    
We used these variables from the data with the goal of identifying any underlying correlations or biases in the data.

### Data Extraction

Our data source provided the data as an RDA file. To convert this data to a CSV, we ran the following in RStudio, where "37990-0001-Data2.rda" was the downloaded RDA file and da37990.0001 was the dataframe in R:

```
write.csv(da37990.0001, "/Users/.../Downloads/37990-0001-Data2.rda\\criminalData.csv", row.names=TRUE)
```
Then, as a CSV, we imported the data into the Jupyter Notebook, as shown below.

In [ ]:
import pandas as pd

df_crime = pd.read_csv("37990-0001-Data2.rda\criminalData.csv")

# extract relevant columns
df_crime = df_crime.loc[:, ['AGE','CITIZEN','MONRACE','HISPORIG','MONSEX','TOTPRISN','OFFGUIDE','NOCOUNTS','TOTCHPTS']]

df_crime.dropna(axis=0, inplace=True)

df_crime.head()

### Data Visualization

In [ ]:
import matplotlib.pyplot as plt

# creates dictionary to hold the number of defendants of each race
race_count = dict()
for race in df_crime["MONRACE"]:
    if race not in race_count:
        race_count[race] = 1
    else:
        race_count[race] += 1
        
# creates dictionary to hold the average prison sentence ordered for a defendant of each race
avg_totprisn_by_race = dict()
# fills dictionary with average prison sentence for all defendants of each race
for index in range(len(df_crime)):
    row = df_crime.iloc[index, :]
    if row[2] not in avg_totprisn_by_race.keys():
        avg_totprisn_by_race[row[2]] = row[5]
    else:
        avg_totprisn_by_race[row[2]] += row[5]
        
for race in avg_totprisn_by_race.keys():
    avg_totprisn_by_race[race] = avg_totprisn_by_race[race] / race_count[race]
    
list_values = sorted(list(avg_totprisn_by_race.values()), reverse=True)

new_dict = dict()
key_list = list(avg_totprisn_by_race.keys())
val_list = list(avg_totprisn_by_race.values())

for val in list_values:
    new_dict[key_list[val_list.index(val)]] = val

# shortens chracter count for labels in bar graph and plots the graph
x_labels = dict()
for race in new_dict.keys():
    x_labels[race] = race[:31]    

plt.bar(x_labels.values(), new_dict.values())    
plt.xticks(rotation = 90)
plt.xlabel("Races")
plt.ylabel("Average Total Prison Time Ordered")
plt.title("Average Total Prison Time Ordered by Race")
plt.show()

<a id='bias'></a>
### Ethical Considerations

The dataset being used is inherently biased due to the disparities concerning race, ethnicity, age, sex, citizenship, etc. that pervade the U.S. criminal justice system. Racial discrimination is the most noteworthy form of bias in our data; the distribution of total prison time for minority populations is much higher compared to the total prison time of Caucasians. Systematic racial discrimination permeates every stage of the United States criminal justice system including disproportionate policing in minority neighborhoods, unfair detainment prior to trial, and unequal parole surveillance. Because of thIS inherent bias, any conclusions made using this dataset will lead to unfair outcomes for individuals who belong to certain demographic groups which can also reinforce stereotypes in society that can lead to further discrimination and marginalization. Lastly, using this dataset to train our linear regression model will lead to the amplification of aforementioned biases and inaccurate predictions about which defendants are likely to re-offend or pose a danger to society, potentially resulting in unjustified restrictions on individuals' freedoms.
Therefore, we recognize that any result that is to be derived from this project will not be an accurate measure of criminal sentencing time, and the sole purpose of our project is to identify and highlight bias in the federal criminal justice system. 

## Method

In order to identify discrepancies in criminal justice sentencing, a multiple regression model was used, as it provides a simple and efficient way to examine the relationship between the independent and dependent variables. Calculating the R² value of the regression model, we are able to evaluate the extent to which the chosen regressors explain the dependent variable. Furthermore, because the coefficients of each regressor represent the direction and magnitude of the relationship between the variables, we are able to properly identify the significance and weightage of each variable in calculating the prison time.  

We created two main regression models, one including the variables for race and one without, with the goal of identifying any differences in the R² value. If the regression model without race decreases the R² value, it means that race did influence the total prison time. Ideally, the R² values should be the same, indicating that race is not a factor that should be considered when determining total prison time.

One of the main setbacks to our project was that the data was mainly categorical, limiting the possibilities of tools that would properly be able to identify how biases are introduced into the dataset. In order to convert our categorical data into numerical data, we used the Pandas get_dummies function, which turned the categorical data into a set of binary columns, also known as dummy variables.

In [ ]:
df_crime = pd.get_dummies(df_crime, columns=['CITIZEN', 'MONRACE', 'HISPORIG','MONSEX','OFFGUIDE'])
df_crime

### Limitations

A main concern in our regression model is omitted variable bias, in which variables not included can potentially skew the results of the regression analysis. In order to make our model stronger, we kept all potential explanatory variables in our regression such as race, citizenship status, and age, and had our dependent variable as prison time in months. We must also note that the limited number of observations affects the value and sign of the coefficients, as the information available for each defendant in the data set varies for each category, and could potentially skew our results. Furthermore, due to biases in data collection, we chose against cross validating our model as splitting the data could result in skewed proportions of defendants with certain characteristics. Additionally, using biased data to train our model will lead it to making even more biased predictions.

## Results

In [ ]:
# Citation: Based on day16_sec4 notes by Professor Higger.

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

def disp_regress(df, x_feat_list, y_feat, verbose=True):
    """ linear regression, displays model w/ coef
    
    Args:
        df (pd.DataFrame): dataframe
        x_feat_list (list): list of all features in model
        y_feat (list): target feature
        verbose (bool): toggles command line output
        
    Returns:
        reg (LinearRegression): model fit to data
    """
    # initialize regression object
    reg = LinearRegression()

    # get target variable
    # (note: since we index with list -> garauanteed 2d x array, no reshape needed)
    x = df.loc[:, x_feat_list].values
    y = df.loc[:, y_feat].values

    # fit regression
    reg.fit(x, y)

    # compute / store r2
    y_pred = reg.predict(x)
    
    bar_chart_dict = dict()
    if verbose:
        # print model   
        model_str = y_feat + f' = {reg.intercept_:.2f}'
        for feat, coef in zip(x_feat_list, reg.coef_):
            bar_chart_dict[feat] = coef
            s_sign = ' - ' if coef < 0 else ' + '
            model_str += s_sign + f'{np.abs(coef):.2f} {feat}'
        print(model_str)

        # compute / print r2
        r2 = r2_score(y_true=y, y_pred=y_pred)
        print(f'r2 = {r2:.5}')
    
    return bar_chart_dict

In [ ]:
# creates the first model with all variables
bar_chart_dict = disp_regress(df=df_crime, y_feat='TOTPRISN', x_feat_list=['AGE', 'NOCOUNTS', 'TOTCHPTS',
       'CITIZEN_(1) United States citizen', 'CITIZEN_(2) Resident/legal alien', 'CITIZEN_(3) Illegal alien',
       'CITIZEN_(4) Not a US citizen/alien status unknown', 'CITIZEN_(5) Extradited Alien', 
       'MONRACE_(01) White / Caucasian', 'MONRACE_(02) Black / African-American', 
       'MONRACE_(03) American Indian or Alaskan Native', 'MONRACE_(04) Asian or Pacific Islander', 
       'MONRACE_(05) Multi-racial', 'MONRACE_(07) Other',
       'MONRACE_(08) Info on race not available in Docs (This code only available in FY07 and on)',
       'MONRACE_(09) Non-US American Indians',
       'HISPORIG_(0) Information on Hispanic Origin Not Available',
       'HISPORIG_(1) Non-Hispanic', 'HISPORIG_(2) Hispanic', 
       'MONSEX_(0) Male', 'MONSEX_(1) Female', 
       'OFFGUIDE_(01) Administration of Justice', 'OFFGUIDE_(02) Antitrust', 'OFFGUIDE_(03) Arson',
       'OFFGUIDE_(04) Assault', 'OFFGUIDE_(05) Bribery/Corruption', 'OFFGUIDE_(06) Burglary/Trespass', 
       'OFFGUIDE_(07) Child Pornography', 'OFFGUIDE_(08) Commercialized Vice', 'OFFGUIDE_(09) Drug Possession',
       'OFFGUIDE_(10) Drug Trafficking', 'OFFGUIDE_(11) Environmental', 'OFFGUIDE_(12) Extortion/Racketeering', 
       'OFFGUIDE_(13) Firearms', 'OFFGUIDE_(14) Food and Drug', 'OFFGUIDE_(15) Forgery/Counter/Copyright',
       'OFFGUIDE_(16) Fraud/Theft/Embezzlement', 'OFFGUIDE_(17) Immigration', 'OFFGUIDE_(18) Individual Rights', 
       'OFFGUIDE_(19) Kidnapping', 'OFFGUIDE_(20) Manslaughter', 'OFFGUIDE_(21) Money Launder',
       'OFFGUIDE_(22) Murder', 'OFFGUIDE_(23) National Defense', 'OFFGUIDE_(24) Obscenity/Other Sex Offenses',
       'OFFGUIDE_(25) Prison Offenses', 'OFFGUIDE_(26) Robbery', 'OFFGUIDE_(27) Sex Abuse', 
       'OFFGUIDE_(28) Stalking/Harassing', 'OFFGUIDE_(29) Tax', 'OFFGUIDE_(30) Other'])

After developing the first multiple regression model, the resulting regression equation is as follows with R² = 0.11927. The names of categories have been redacted for conciseness, but can be found above.

$ TOTPRISN = 44.00 + 0.79 AGE + 30.16 NOCOUNTS + 4.02 TOTCHPTS - 12.77 CITIZEN_(1) - 15.22 CITIZEN_(2) - 10.21 CITIZEN_(3) - 0.88 CITIZEN_(4) + 39.08 CITIZEN_(5) - 1.36 MONRACE_(01) + 6.66 MONRACE_(02) - 31.39 MONRACE_(03) + 13.02 MONRACE_(04) + 6.07 MONRACE_(05) + 24.83 MONRACE_(07) - 11.51 MONRACE_(08) - 6.32 MONRACE_(09) - 7.64 HISPORIG_(0) + 4.50 HISPORIG_(1) + 3.14 HISPORIG_(2) + 8.50 MONSEX_(0) - 8.50 MONSEX_(1) - 104.93 OFFGUIDE_(01) - 107.36 OFFGUIDE_(02) - 69.06 OFFGUIDE_(03) - 21.05 OFFGUIDE_(04) - 127.69 OFFGUIDE_(05) - 100.56 OFFGUIDE_(06) + 0.26 OFFGUIDE_(07) - 113.62 OFFGUIDE_(08) - 112.56 OFFGUIDE_(09) - 41.00 OFFGUIDE_(10) - 117.85 OFFGUIDE_(11) - 105.71 OFFGUIDE_(12) - 91.38 OFFGUIDE_(13) - 107.14 OFFGUIDE_(14) - 133.09 OFFGUIDE_(15) - 118.02 OFFGUIDE_(16) - 106.36 OFFGUIDE_(17) - 105.84 OFFGUIDE_(18) + 574.02 OFFGUIDE_(19) - 26.42 OFFGUIDE_(20) - 35.61 OFFGUIDE_(21) + 1650.48 OFFGUIDE_(22) - 80.02 OFFGUIDE_(23) - 114.97 OFFGUIDE_(24) - 114.77 OFFGUIDE_(25) - 49.64 OFFGUIDE_(26) + 248.96 OFFGUIDE_(27) - 115.47 OFFGUIDE_(28) - 142.10 OFFGUIDE_(29) - 111.52 OFFGUIDE_(30) $ 

Because the model has multiple independent variables and only one dependent variable (total prison time in months), the model can not be graphed like a standard regression model. So, to visualize the results, bar charts were created to compare the model’s coefficients for citizenship status, race, Hispanic origin, and type of crime. Coefficients with positive (or negative) values indicate an increase (or decrease) in the amount of total prison time predicted for defendants with that given trait. Thus, by comparing the direction and magnitude of various independent variables’ coefficients, biases in the data may be highlighted. The graphs below indicate the coefficients for the stated independent variables.

<a id='graphs'></a>

In [ ]:
# create initial dicts to store bar chart data in
citizen = dict()
race = dict()
offguide = dict()
hisporig = dict()

# add coefficient and variable name for each category in respective dictionary
for key, value in bar_chart_dict.items():
    if 'MONRACE' in key:
        race[key] = round(value, 2)
    elif 'CITIZEN' in key:
        citizen[key] = round(value, 2)
    elif 'OFFGUIDE' in key:
        offguide[key] = round(value, 1)
    elif 'HISPORIG' in key:
        hisporig[key] = round(value, 2)

# used to label coefficient values for each bar in chart
def addlabels(x,y):
    for i in range(len(x)):
        if x==list(hisporig.keys()):
            plt.text(i, y[i]+0.15, y[i], ha = 'center')
        else:
            plt.text(i, y[i]+1, y[i], ha = 'center')

# create dicts to store category dicts and color info
category_dict = {'race': race, 'citizen': citizen, 'hisporig': hisporig,'offguide':offguide}
color_dict = {'race':'tan','citizen':'lightsteelblue','hisporig':'rosybrown','offguide':'thistle'}

# go through each dictionary name and dictionary
for dict_name, category in category_dict.items():
    
    # extract specific groups in category for bar chart labels
    x_labels = list()
    for key in category.keys():
        x_labels.append(key[:31])
        
    # plot formatting for all bar charts
    plt.bar(x_labels, category.values(), color = color_dict[dict_name])
    addlabels(list(category.keys()), list(category.values()))
    plt.xticks(rotation=90)

    # plot formatting specific to each graph
    if dict_name == 'race':
        plt.xlabel("DEFENDANT'S RACE")
        plt.ylabel("COEFFICIENT IN REGRESSION MODEL")
        plt.gcf().set_size_inches(8,8)
        plt.title("Figure 1")
        plt.show()
        
    if dict_name == 'citizen':
        plt.xlabel("DEFENDANT'S CITIZENSHIP STATUS")
        plt.ylabel("COEFFICIENT IN REGRESSION MODEL")
        plt.gcf().set_size_inches(8,8)
        plt.title("Figure 2")
        plt.show()
        
    if dict_name == 'hisporig':
        plt.xlabel("DEFENDANT'S HISPANIC ORIGIN")
        plt.ylabel("COEFFICIENT IN REGRESSION MODEL")
        plt.gcf().set_size_inches(6,6)
        plt.title("Figure 3")
        plt.show()
    
    if dict_name == 'offguide':
        plt.xlabel("TYPE OF CRIMINAL OFFENSE")
        plt.ylabel("COEFFICIENT IN REGRESSION MODEL")
        plt.gcf().set_size_inches(20,20)
        plt.title("Figure 4")
        plt.show()

For the second multiple regression model (without race as an independent variable), the resulting regression equation is as follows with R² = 0.11911.

In [ ]:
bar_chart_dict2 = disp_regress(df=df_crime, y_feat='TOTPRISN', x_feat_list=['AGE', 'NOCOUNTS', 'TOTCHPTS',
       'CITIZEN_(1) United States citizen', 'CITIZEN_(2) Resident/legal alien',
       'CITIZEN_(3) Illegal alien',
       'CITIZEN_(4) Not a US citizen/alien status unknown',
       'CITIZEN_(5) Extradited Alien',
       'HISPORIG_(0) Information on Hispanic Origin Not Available',
       'HISPORIG_(1) Non-Hispanic', 'HISPORIG_(2) Hispanic', 'MONSEX_(0) Male',
       'MONSEX_(1) Female', 'OFFGUIDE_(01) Administration of Justice',
       'OFFGUIDE_(02) Antitrust', 'OFFGUIDE_(03) Arson',
       'OFFGUIDE_(04) Assault', 'OFFGUIDE_(05) Bribery/Corruption',
       'OFFGUIDE_(06) Burglary/Trespass', 'OFFGUIDE_(07) Child Pornography',
       'OFFGUIDE_(08) Commercialized Vice', 'OFFGUIDE_(09) Drug Possession',
       'OFFGUIDE_(10) Drug Trafficking', 'OFFGUIDE_(11) Environmental',
       'OFFGUIDE_(12) Extortion/Racketeering', 'OFFGUIDE_(13) Firearms',
       'OFFGUIDE_(14) Food and Drug',
       'OFFGUIDE_(15) Forgery/Counter/Copyright',
       'OFFGUIDE_(16) Fraud/Theft/Embezzlement', 'OFFGUIDE_(17) Immigration',
       'OFFGUIDE_(18) Individual Rights', 'OFFGUIDE_(19) Kidnapping',
       'OFFGUIDE_(20) Manslaughter', 'OFFGUIDE_(21) Money Launder',
       'OFFGUIDE_(22) Murder', 'OFFGUIDE_(23) National Defense',
       'OFFGUIDE_(24) Obscenity/Other Sex Offenses',
       'OFFGUIDE_(25) Prison Offenses', 'OFFGUIDE_(26) Robbery',
       'OFFGUIDE_(27) Sex Abuse', 'OFFGUIDE_(28) Stalking/Harassing',
       'OFFGUIDE_(29) Tax', 'OFFGUIDE_(30) Other'])

$ TOTPRISN = 42.24 + 0.77 AGE + 30.28 NOCOUNTS + 4.06 TOTCHPTS - 13.71 CITIZEN_(1) - 14.59 CITIZEN_(2) - 10.69 CITIZEN_(3) - 0.27 CITIZEN_(4) + 39.26 CITIZEN_(5) - 8.74 HISPORIG_(0) + 6.55 HISPORIG_(1) + 2.19 HISPORIG_(2) + 9.07 MONSEX_(0) - 9.07 MONSEX_(1) - 104.73 OFFGUIDE_(01) - 102.74 OFFGUIDE_(02) - 70.15 OFFGUIDE_(03) - 31.85 OFFGUIDE_(04)  - 123.29 OFFGUIDE_(05) - 110.82 OFFGUIDE_(06) + 0.30 OFFGUIDE_(07) - 107.89 OFFGUIDE_(08) - 110.27 OFFGUIDE_(09) - 38.28 OFFGUIDE_(10) - 119.37 OFFGUIDE_(11) - 102.32 OFFGUIDE_(12) - 87.67 OFFGUIDE_(13) - 105.03 OFFGUIDE_(14) - 130.09 OFFGUIDE_(15) - 114.98 OFFGUIDE_(16) - 104.54 OFFGUIDE_(17) - 102.66 OFFGUIDE_(18) + 576.09 OFFGUIDE_(19) - 49.85 OFFGUIDE_(20) - 32.62 OFFGUIDE_(21) + 1650.88 OFFGUIDE_(22) - 77.48 OFFGUIDE_(23) - 118.85 OFFGUIDE_(24) - 113.09 OFFGUIDE_(25) - 45.54 OFFGUIDE_(26) + 246.44 OFFGUIDE_(27) - 119.08 OFFGUIDE_(28) - 138.39 OFFGUIDE_(29) - 112.12 OFFGUIDE_(30) $

Due to insignificant changes in coefficients for the independent variables, the bar charts for coefficient comparison were omitted for this model.

## Discussion

### Interpretation of Results

The two regression models produced R² values of 0.11927 and 0.11911, respectively. These are relatively low values, indicating that our model isn't entirely effective. Because of the volatility of our dataset, it makes sense that relationships between variables could not be fully captured by this model, hence, resulting in low R² values. 

Now, examining the relationship between prison time and race in the dataset in Figure 1, we can see that White defendants tend to have an increase of only 1 month in total prison time, compared to that of Black defendants, Asian/Pacific Islander defendants,  or defendants of other races who have increases of 7, 13, and 25 months in prison time, respectively. This may indicate biases in the data as we can see clear differences in the coefficients (and thus prison times) for different races. 

We can also see the coefficient for American Indian or Alaskan Native defendants is very low with a value of about -31, meaning defendants of this race tend to have about 31 months less of prison time, which is particularly surprising due to the fact that defendants of this race had the highest average prison time, according to the earlier bar graph of average total prison time by race. This points to errors within our ML methods, which may be attributed to American Indian or Alaskan Native defendants with high prison times, which would skew prison time averages.

Figure 2 shows the coefficient values for citizenship status. It is evident that those labeled as extradited aliens are likely to have a higher prison sentencing, indicated by the positive coefficient value. The other variables, such as United States citizen, resident, illegal alien, and not a US citizen are negatively correlated with total prison time, meaning that being in one of those categories would generally indicate a lower prison time. Although our model may be skewed, it is generally intuitive that those labeled as extradited aliens are likely to face a higher prison sentencing in comparison to those who are US citizens.

In Figure 3, the coefficient with the greatest magnitude was for the group where information on Hispanic origin was not available. Therefore, our results for this category were generally inconclusive.

Finally, looking at the relationship between the type of criminal offense and prison time in Figure 4, it is evident that the two highest bars belong to more serious offenses, including murder and kidnapping. This makes sense in terms of our dataset, as it acknowledges the relationship between crime severity and prison time. Additionally, because of the vast difference in prison time between serious offenses and other offenses, it is difficult to note significant differences between the coefficients for other offenses. These severe crimes may, thus, serve as outliers and skew the data as well.

<a id='takeaway'></a>
### Takeaways and Future Implications

Overall, our model definitely carried errors in terms of outliers and skewed coefficients, but from what we examined, we noted that defendants of certain races corresponded to higher prison times, on average, indicating possible biases. We also saw that type of crime is a large factor in deciding prison time, as one would expect. In summation, the most salient ethical implication of our project was the presence of possible racial bias in federal criminal sentencing. Additionally, due to the biased nature of our dataset, the results derived from our project should not be used for any kind of prediction and should only be used to highlight biases in the system. 

Further work on this topic could include developing a random forest classifier and examining the resulting feature importance graph, or analyzing datasets with more independent variables that could affect a defendant’s prison time. If biases are continued to be shown in future data analysis research, policies to relieve marginalized groups from longer prison sentences should be implemented at multiple levels including data collection methods, revising sentencing guidelines, and training and educating court personnel.